To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News


Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

[gpt-oss RL](https://docs.unsloth.ai/new/gpt-oss-reinforcement-learning) is now supported with the fastest inference & lowest VRAM. Try our [new notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/gpt-oss-(20B)-GRPO.ipynb) which creates kernels!

Introducing [Vision](https://docs.unsloth.ai/new/vision-reinforcement-learning-vlm-rl) and [Standby](https://docs.unsloth.ai/basics/memory-efficient-rl) for RL! Train Qwen, Gemma etc. VLMs with GSPO - even faster with less VRAM.

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
%%capture
import os, re

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch
    v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")

    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
    !pip install transformers==4.56.2  # novější kompatibilní verze

### Unsloth

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#### Text Completion / Raw Text Training



In [3]:
%env UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT

env: UNSLOTH_RETURN_LOGITS=1 # Run this to disable CCE since it is not supported for CPT


In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",      # New Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",           # Llama-3 15 trillion tokens model 2x faster!
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",        # Phi-3 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",             # Gemma 2.2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct", # "unsloth/mistral-7b" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.2: Fast Mistral patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

We also add `embed_tokens` and `lm_head` to allow the model to learn out of distribution data.

In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.11.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


<a name="Data"></a>
### Data Prep

In [6]:
import re

# Downloading dataset
!wget -nc https://raw.githubusercontent.com/valkova-k/cactus-repo/refs/heads/main/assignment05/combined_books.txt

pattern = r'(\.\s*\.\s*\.\s*\d+)|([–-]\s*\d+\s*[–-])'

with open("combined_books.txt", "r", encoding="utf-8") as f:
    raw_text = "".join(line for line in f if not re.search(pattern, line))

--2025-11-09 14:31:14--  https://raw.githubusercontent.com/valkova-k/cactus-repo/refs/heads/main/assignment05/combined_books.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2588154 (2.5M) [text/plain]
Saving to: ‘combined_books.txt’

combined_books.txt  100%[===================>]   2.47M  --.-KB/s    in 0.05s   

2025-11-09 14:31:14 (52.7 MB/s) - ‘combined_books.txt’ saved [2588154/2588154]



In [7]:
print("length of dataset in characters: ", len(raw_text))
print(raw_text[:5000])

length of dataset in characters:  2304298
Produced by Miloslav Izar RUSKÁ KNIHOVNA IX. SPISY FEDORA MICHAJLOVIČE DOSTOJEVSKÉHO. Překlad rediguje JAROMÍR HRUBÝ Svazek I. ZÁPISKY Z MRTVÉHO DOMU. Přeložil H. JAROŠ. V PRAZE 1891. Tiskem a nákladem J. Otty. ČÁST PRVÁ. ÚVOD. V dalekých krajích Sibiře, uprostřed stepí, hor a neproniknutelných lesů vyskytují se zřídka malinká města s jedním nebo nanejvýš se dvěma tisíci obyvatelů, dřevěná to, neúhledná města se dvěma chrámy, jedním ve městě, druhým na hřbitově, a podobná více k slušné vesnici pod Moskvou než k městu. Bývají obyčejně hojně opatřena policejními hejtmany, komisary a ostatními podřízenými policejními dozorci. V Sibiři vůbec přes to, že je tam zima. jsou úřady neobyčejně teploučké. Lid tamní je prostý, nenačichlý liberálností; pořádky staré, pevné, staletími posvěcené. Úředníky, kteří právem hrají úlohu sibiřské šlechty, jsou buď tuzemci, zakořenělí Sibiřáci, anebo rodáci z evropského Ruska, zejména hlavních měst, kteří se dali při

In [16]:
raw_text = re.sub(r"\s+", " ", raw_text).strip()

print("Délka celého textu ve znacích:", len(raw_text))

from datasets import Dataset

sentences = re.split(r'(?<=[.!?])\s+', raw_text)

print("Počet vět:", len(sentences))
print("Ukázka prvních 5 vět:\n", sentences[:5])

# --- spojování vět do chunků po 10 větách ---
chunk_size = 15  # můžeš změnit na 5 nebo 15 podle potřeby
chunks = [
    " ".join(sentences[i:i + chunk_size])
    for i in range(0, len(sentences), chunk_size)
    if len(sentences[i:i + chunk_size]) > 0
]

print("Počet chunků:", len(chunks))
print("Ukázka prvního chunku:\n", chunks[0][:500])

# --- vytvoření datasetu pro UnSloth ---
EOS_TOKEN = tokenizer.eos_token
texts_for_training = [c + EOS_TOKEN for c in chunks]

dataset = Dataset.from_dict({"text": texts_for_training})
print(dataset)
print("První záznam v datasetu:\n", dataset[2]["text"])

Délka celého textu ve znacích: 2302653
Počet vět: 24401
Ukázka prvních 5 vět:
 ['Produced by Miloslav Izar RUSKÁ KNIHOVNA IX.', 'SPISY FEDORA MICHAJLOVIČE DOSTOJEVSKÉHO.', 'Překlad rediguje JAROMÍR HRUBÝ Svazek I.', 'ZÁPISKY Z MRTVÉHO DOMU.', 'Přeložil H.']
Počet chunků: 1627
Ukázka prvního chunku:
 Produced by Miloslav Izar RUSKÁ KNIHOVNA IX. SPISY FEDORA MICHAJLOVIČE DOSTOJEVSKÉHO. Překlad rediguje JAROMÍR HRUBÝ Svazek I. ZÁPISKY Z MRTVÉHO DOMU. Přeložil H. JAROŠ. V PRAZE 1891. Tiskem a nákladem J. Otty. ČÁST PRVÁ. ÚVOD. V dalekých krajích Sibiře, uprostřed stepí, hor a neproniknutelných lesů vyskytují se zřídka malinká města s jedním nebo nanejvýš se dvěma tisíci obyvatelů, dřevěná to, neúhledná města se dvěma chrámy, jedním ve městě, druhým na hřbitově, a podobná více k slušné vesnici p
Dataset({
    features: ['text'],
    num_rows: 1627
})
První záznam v datasetu:
 A v Sibiři jí užívati umějí. V jednom z takových veselých, sebou spokojených městeček s roztomilým obyvatelstvem, je

<a name="Train"></a>
### Continued Pretraining
Now let's use Unsloth's `UnslothTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 20 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

Also set `embedding_learning_rate` to be a learning rate at least 2x or 10x smaller than `learning_rate` to make continual pretraining work!

In [17]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,   # stačí 1, ať se to nehádá

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,

        warmup_ratio = 0.02,
        num_train_epochs = 3,      # místo 100!

        learning_rate = 1e-5,      # jemnější LR
        embedding_learning_rate = 1e-6,

        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.00,
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/1627 [00:00<?, ? examples/s]

In [18]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.941 GB of memory reserved.


In [19]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,627 | Num Epochs = 3 | Total steps = 306
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 8 x 1) = 16
 "-____-"     Trainable parameters = 436,076,544 of 4,257,156,096 (10.24% trained)


Step,Training Loss
10,3.245100
20,3.189000
30,3.154200
40,3.161800
50,3.115200
60,3.073400
70,3.106100
80,3.083600
90,3.067000
100,3.029100


In [20]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

6396.0321 seconds used for training.
106.6 minutes used for training.
Peak reserved memory = 9.625 GB.
Peak reserved memory for training = 1.684 GB.
Peak reserved memory % of max memory = 65.294 %.
Peak reserved memory for training % of max memory = 11.424 %.


<a name="Inference"></a>
### Inference
Let's run the model!

In [24]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

from transformers import TextIteratorStreamer  # už vlastně nepotřebujeme
import torch

# přepnout model do inference režimu
FastLanguageModel.for_inference(model)

seeds = [
    "Člověk je zvláštní tvor, pomyslel si Raskolnikov. ",
    "„Ty tomu nerozumíš,“ řekl tiše. ",
    "Ráno se probudil se zvláštním pocitem neklidu.",
]

device = "cuda" if torch.cuda.is_available() else "cpu"

for seed in seeds:
    print(f"\n\n=== Prompt: {seed} ===\n")

    inputs = tokenizer(seed, return_tensors="pt").to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens = 256,
        do_sample = True,
        temperature = 0.8,
        top_p = 0.9,
        repetition_penalty = 1.1,
    )

    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(text)
    print("\n" + "-"*80)




=== Prompt: Člověk je zvláštní tvor, pomyslel si Raskolnikov.  ===

Člověk je zvláštní tvor, pomyslel si Raskolnikov. ‚To jsou časy!‘ podotkl ze židiny k nám přece z hlavy: všecko bude včera jít po svém domácímu a posléze na věci přednostné, ale večer… Víte-li, což jsem stále slyšel za okamžik? Ale o tom ho dávno neviděl ani jeden krátkodenný můj muže, jak by byl znal jeho výsledek i jeho příjemnosti, když měl vaší dob, kdy se myslil ubohým rozumem, že oni nemají dost potřebu proti všemu! Avtovák byl tak zajímavý – považme to ještě jediné, kde máme dobrého času — já tu však odpustila pozorování. Je mi naprosto pranic, že proto jsem teprve jako nové šaty přiznal nějakou nejrozumnější, netradičnou radostnoutu formu; ale také pak bylo totiž bezpochyby,

--------------------------------------------------------------------------------


=== Prompt: „Ty tomu nerozumíš,“ řekl tiše.  ===

„Ty tomu nerozumíš,“ řekl tiše. Říkání se tak obtěžoval na rukou kůňi. Byla hned přece v něm jasna a čer

## Výstupy z GPT2 se stejnými prompty

=== SEED: 'Člověk je zvláštní tvor, pomyslel si Raskolnikov. ' ===

Člověk je zvláštní tvor, pomyslel si Raskolnikov.  Zatraceně  dokud
chvíli. „Tak, že jste vrah!“ začal prohlédl, „že tam?
že jsi děláš?“
„To je to všechno vůbec neptávám nikoho?“ vyhrklínal Razumichin.
„Ale kdy já nevyzpyt?“
„Jak to ochotě toho nedbát ostatně bytná, že známý… Ano!“
„Já není, protože vjete jsem mě, může jste se možná zákon-
na už to však vás bytím,“ řekla v něm, „ěmž i není vám jste
zapomenoutné… jsem jim to tady jen teď, protože je tak… ná, já jsem se mi
mluvím pravdu… Tady tady nejvím, bude nevím tak přímo…“
vždyť jsem začal, jak skutečně dopadne rozpaků, ale jsem se dal
se to zakrytovat. Už jsem si s ním jí začala, že
tak zahlrát řeči… he

=== SEED: '„Ty tomu nerozumíš,“ řekl tiše. ' ===

„Ty tomu nerozumíš,“ řekl tiše.  „Kdo věci  vám  že  už  náležitost  odpuštěv,
protože sešel oči. Zkrátil jeden s takovým přímo po černém
a přesvědčením a kapsy si na to. A protože si opravdí se tak příliš
dokonce náhle za něco přítati, o čemu si pro mne přijdu.
„Nebude svých se nebýt za dávno,“ řekl Raskolnikov. „Kdybych si však neměl! Jsem
dobr mou duševně.“
„A ty jsem řekl, že jsem vzpomněl jsem přesvědčen,“ po-
chin, „jak jsem tak zvalo, že hned počal,“ umlčel na ni zá-
místa obtím nedá třeje a s netrpělivě Ivanovně odhaleně ze
nává, kterou jsem tak řekl, že jsem se to říkával, že se
už ho říkám, že by mu to předtím.“
„No, v

=== SEED: 'Ráno se probudil se zvláštním pocitem neklidu.' ===

Ráno se probudil se zvláštním pocitem neklidu. Ostatně bylo možno státi. A vyskytl si dokonce nejvíce buldruším o tom, že se zdálo, že se mu rozdálo: opakoval, že snažil na místa, tak taková vězněsíce. Už není nic nikterak neprosíce nevadí. Zde mně dopustím vyhledával se, že ji porozumí, málo pro tebe, a bylo představiti, co se oblékávati. Zdá se, že bylo za to, co tak já, že nemožná dosti představení prostraní, aby se však síluším vás velikých a skoro směšností, a že jsem sám představenstvo. Zde on sám s nám v nemocnicí. Ž...ský trestanci, za narazil jsem úplně dva zapomněl. Ale vášeli, že pan Goljadkin mladší síla na jevo, náhle se nazpokojila s podnik, jakoby byl vědom vlastně spolehlivé sluha, které nemohu počínalo se mně s nimiž mysli. „Ano, �
